## Generator imion, nazwisk, e-mailu, numerów telefonów pracowników

### 1)Odczytywanie popularnych imion i nazwisk żeńskich i męskich.

In [90]:
import pandas as pd
import pathlib as path


# Znajdź katalog z imionami i nazwiskami.
name_dir_path: path.Path = path.Path().cwd()/"Imiona i nazwiska"

def read_names(first_names_file:str, last_names_file:str, dir_path:path.Path) -> tuple[pd.Series, pd.Series]:
    """Funkcja odczytuje pliki zawierające najpopularniejsze polskie imiona i nazwiska. Wczytuje je do szeregu danych typu pandas"""
    # Odczytaj imiona oraz nazwiska.
    first_names: pd.DataFrame = pd.read_excel(dir_path/first_names_file  ,"Arkusz1",
                                                usecols = [0,2]
                                                )
    last_names: pd.DataFrame = pd.read_excel(dir_path/last_names_file, "Arkusz1",
                                                nrows = 5_000) # Wierszy w arkuszu jest bardzo dużo (ponad 380 000). 
                                                                # Odczytaj tylko pewną część wierszy.
                                                                

    # Bierzemy pod uwagę jedynie najpopularniejsze imiona i nazwiska.
    first_names = first_names.loc[first_names["LICZBA_WYSTĄPIEŃ"] > 500, "IMIĘ_PIERWSZE"]
    last_names = last_names.loc[last_names["Liczba"] > 500, "Nazwisko aktualne"] 

    # Skróc nazwy kolumn.
    first_names.columns = ['imię']
    last_names.columns = ['nazwisko']

    return first_names, last_names

# Wczytaj męskie imiona i nazwiska.
male_first_names, male_last_names = read_names("imiona_meskie.xlsx", "nazwiska_meskie.xlsx", name_dir_path)

# Wczytaj żęńskie imiona i nazwiska.
female_first_names, female_last_names = read_names("imiona_zenskie.xlsx", "nazwiska_zenskie.xlsx", name_dir_path)


###  Tworzenie danych osobowych.

In [91]:
import numpy as np

n_female, n_male = 25, 21 # Ustal ile będzie pracowniczek i pracowników.


def generate_personal_data(first_names:pd.Series, last_names:pd.Series, n:int, email_domain:str) -> pd.DataFrame:
    """Funkcja generuje dane osobiste pracowników. Tworzone są pary imię-nazwisko, numer telefonu oraz adres e-mail firmowy.

    """

    # Przygotuj wyjściową ramkę danych.
    workers_df: pd.DataFrame = pd.DataFrame(data = {"imię":pd.Series(dtype = pd.StringDtype()),
                                                    "nazwisko": pd.Series(dtype = pd.StringDtype()), 
                                                    "numer telefonu":pd.Series(dtype = pd.StringDtype()), 
                                                    "adres email": pd.Series(dtype = pd.StringDtype())},
                                                
    )

    # Lista wszystkich cyfr. Przyda się przy generowaniu numerów telefonów.
    digits:list[str] = ["0","1","2","3","4","5","6","7","8","9"]

    # Stwórz n fikcyjnych różnych osób.
    for id in range(n):
        rand_first_name_idx:int = np.random.randint(0, first_names.shape[0])
        rand_last_name_idx:int = np.random.randint(0, last_names.shape[0])

        rand_first_name:str = first_names[rand_first_name_idx] # Wylosuj imię
        rand_last_name:str = last_names[rand_last_name_idx]  # Wylosuj nazwisko.

        # Wygeneruj losowy numer telefonu nie zaczynający się od 0.
        phone_number:np.array = np.random.choice(digits, 9)

        # Jeżeli wylosowaliśmy zero, podmień cyfrę.
        if phone_number[0] == "0":
            phone_number[0] = np.random.choice(digits[1:], 1)[0]

        phone_number: str = "".join(phone_number)

        # Stwórz email dla tego pracownika
        email:str = rand_first_name.lower() + rand_last_name.lower() + "".join(np.random.choice(digits, 3)) +"@" + email_domain
        

        workers_df.loc[id, :] = [rand_first_name, rand_last_name, phone_number, email]

    return workers_df


        

# Wygeneruj dane osobiste dla pracowniczek
email_domain = "nowowiejska.com"
n_male, n_female = 15, 25

female_workers_data:pd.DataFrame = generate_personal_data(female_first_names, female_last_names, n_female, email_domain)
male_workers_data: pd.DataFrame = generate_personal_data(male_first_names, male_last_names, n_male, email_domain)

# Połącz kobiecych i męskich pracowników.
workers_data:pd.DataFrame = pd.concat(objs = [female_workers_data, male_workers_data], axis = 0).reset_index(drop = True)

workers_data.insert(0, "ID", range(1, n_female+n_male+1))

workers_data

,ID,imię,nazwisko,numer telefonu,adres email
0,1,SARA,JĘDRZEJAK,956758224,sarajędrzejak514@nowowiejska.com
1,2,LAURA,KLUK,549656676,laurakluk408@nowowiejska.com
2,3,GABRIELA,GOŁĘBIOWSKA,227533429,gabrielagołębiowska922@nowowiejska.com
3,4,LIWIA,DRZYZGA,825981481,liwiadrzyzga908@nowowiejska.com
4,5,LAURA,DRĄŻEK,837785869,lauradrążek753@nowowiejska.com
5,6,ZUZANNA,BOMBA,175087119,zuzannabomba542@nowowiejska.com
6,7,NADIA,SZOŁTYSIK,220965674,nadiaszołtysik458@nowowiejska.com
7,8,OLIWIA,OKUNIEWSKA,220033428,oliwiaokuniewska021@nowowiejska.com
8,9,ANNA,PISKORSKA,540085248,annapiskorska411@nowowiejska.com
9,10,RÓŻA,LUBECKA,623971915,różalubecka021@nowowiejska.com
